In [1]:
import sys
sys.path.append('../../30_data_tools/')
sys.path.append('../process_masks/')

In [2]:
from PIL import Image, ImageOps, ImageFilter, ImageEnhance
import numpy as np
import cv2
import random
import plotly.express as px
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
import pickle
import re
import json
import sqlite3
from datetime import datetime
from skimage import transform
from PIL import ImageDraw, ImageFont
from time import time
import math
from io import BytesIO
from file_interaction import upload_buffer
from generic_image_db_interaction import save_generic_image

In [3]:
from helper import load_dotenv, get_pdf_page_processing_status
from get_labelstudio_data import get_moires_of_project, get_results_of_project

In [4]:
from mask_functions import get_config as get_mask_config, load_masks, get_whole_mask
from get_sample_scores import get_images_with_scores

In [5]:
dotenv = load_dotenv()
mask_config = get_mask_config()

In [6]:
DEBUG = True
N = 5

In [7]:
with open('./configurations/config_pullover.json') as config_file:
    config = json.load(config_file)

In [8]:
con = sqlite3.connect( dotenv['DB_PATH'] )

In [9]:
pdf_pages = pd.merge(
    pd.read_sql(
        '''
            SELECT * FROM pdf_page
            WHERE job = '24-03-05-01_randomTrainPages'
        ''',
        con
    ),
    get_pdf_page_processing_status(
        'halftone600dpi',
        '4c'
    ).loc[
        :,
        ['filename','job','file_available']
    ],
    how='left',
    on=['job','filename']
).rename(columns={'file_available':'halftone_available'})

pdf_pages = pd.merge(
    pdf_pages,
    get_pdf_page_processing_status(
        'halftone600dpi',
        'masks'
    ).loc[
        :,
        ['filename','job','file_available']
    ],
    how='left',
    on=['job','filename']
).rename(columns={'file_available':'masks_available'})

In [10]:
pdf_pages = pdf_pages.loc[
    (pdf_pages.halftone_available) &
    (pdf_pages.masks_available)
]

In [11]:
pdf_pages.shape

(1999, 7)

# Funktionen

In [12]:
def get_next_idx_for_image( row, method ):
    c = con.cursor()
    c.execute(
        f'''
            SELECT IFNULL(max(idx),0) + 1 AS next_idx FROM generic_image gi 
            WHERE 
            	job='{ row.job }' AND 
            	pdf_filename = '{ row.filename }' AND 
            	variant_name = 'halftone{ dotenv["LOFI_DPI"]}dpi' AND 
            	"type" = '4c' AND 
            	"method" = '{ method }' 
        '''
    )
    
    return c.fetchone()[0]

In [13]:
def get_generic_image_name( row, method, next_idx ):
    return f"{ row.job }.{ row.filename }.halftone{ dotenv['LOFI_DPI'] }dpi.{ method }.{ next_idx }.jpg"

In [14]:
def get_random_masks( masks, weights, k ):
    out = []

    while len(out) < k and len(out) < len(masks):
        next_item = random.choices(
            masks,
            weights= weights,
            k=1
        )[0]

        if next_item['bbox'] not in [el['bbox'] for el in out]:
            out.append(next_item)

    return out

In [15]:
def log( message ):
    if DEBUG:
        print( message )

# Ausführung

In [16]:
from pattern_creation import get_pattern_style, get_pattern_img_by_style
from apply_pattern import apply_pattern

In [17]:
from file_interaction import get_related_filepath, open_img

In [18]:
for j in tqdm(range(N)):
    page_row = pdf_pages.sample(n=1).iloc[0]
    
    img_path = get_related_filepath(
        page_row.job,
        f'halftone{ dotenv["LOFI_DPI"] }dpi',
        f'{ page_row.filename }.4c.jpg'
    )
    
    masks_path = get_related_filepath(
        page_row.job,
        f'halftone{ dotenv["LOFI_DPI"] }dpi',
        f'{ page_row.filename }.masks.pkl'
    )
    
    img = open_img( img_path )
    times = []
    times.append((time(),'start'))
    log('start')
    
    # Stimmen Verarbeitungs- und Ausgangsauflösung nicht zusammen
    # werden die Bilder entsprechend skaliert
    if config['processing_dpi'] != dotenv["LOFI_DPI"]:
        img = img.resize(
            (int(img.size[0] * (config['processing_dpi'] / dotenv["LOFI_DPI"])), int(img.size[1] * (config['processing_dpi'] / dotenv["LOFI_DPI"])))
        )
    rows_of_image = []
    
    masks = load_masks( masks_path )
    
    # zu verarbeitende Masken auswählen
    relevant_masks = get_random_masks(
        masks,
        [m['area'] * m['stability_score'] for m in masks],
        config['masks_per_generated_image']
    )
    
    out_img = np.array(img)
    
    times.append((time(),'Vorbereitung abgeschlossen'))
    log(f'{ times[-1][0] - times[0][0] }: Vorbereitung abgeschlossen')
    
    for i in range(len(relevant_masks)):
        m = relevant_masks[i]
    
        # Wenn Verarbeitungs- und Halbtonauflösung abweichen
        # wird die Maske entsprechend skaliert
        if config['processing_dpi'] != dotenv["LOFI_DPI"]:
            mask_scale_factor = config['processing_dpi'] / dotenv["LOFI_DPI"]
            m['mask'] = (cv2.resize(
                (m['mask'] * 255).astype('uint8'),
                (0,0),
                fx=mask_scale_factor,
                fy=mask_scale_factor
            ) / 255).round().astype('bool')
            m['bbox'] = [int(val * mask_scale_factor) for val in m['bbox']]
            
        
        # die verwendeten Anpassungen werden ausgewählt
        row = {
            'job' : page_row.job,
            'pdf_filename' : page_row.filename,
            'method' : 'soft_light', #'screen' if random.random() > 0.5 else 'multiply',
            'ssim' : -1,
            'bbox' : m['bbox'],
            'overlay_intensity_C' : config['overlay_intensity'][0],
            'overlay_intensity_M' : config['overlay_intensity'][1],
            'overlay_intensity_Y' : config['overlay_intensity'][2],
            'overlay_intensity_K' : config['overlay_intensity'][3]
        }
        row['idx'] = get_next_idx_for_image( page_row, row['method'] )
    
        pattern_style = get_pattern_style(config, m)
    
        for key in pattern_style:
            row[key] = pattern_style[key]
    
        pattern_img = get_pattern_img_by_style(row, config)    
        
        times.append((time(), f'Maske geladen'))
        log(f'{ times[-1][0] - times[0][0] }: Maske { i } geladen')
    
        ssim_value_C = 1
        ssim_value_M = 1
        ssim_value_Y = 1
        ssim_value_K = 1
    
        if config['overlay_intensity'][0] > 0:
            out_img[:,:,0],pattern_mask, ssim_value_C = apply_pattern(
                out_img[:,:,0],
                pattern_img,
                m,
                pattern_style['pattern_img_position'],
                config,
                method=row['method'],
                overlay_weight=config['overlay_weight'] * config['overlay_intensity'][0]
            )
    
        if config['overlay_intensity'][1] > 0:
            out_img[:,:,1],pattern_mask, ssim_value_M = apply_pattern(
                out_img[:,:,1],
                pattern_img,
                m,
                pattern_style['pattern_img_position'],
                config,
                method=row['method'],
                overlay_weight=config['overlay_weight'] * config['overlay_intensity'][1],
                log=log
            )
    
        if config['overlay_intensity'][2] > 0:
            out_img[:,:,2],pattern_mask, ssim_value_Y = apply_pattern(
                out_img[:,:,2],
                pattern_img,
                m,
                pattern_style['pattern_img_position'],
                config,
                method=row['method'],
                overlay_weight=config['overlay_weight'] * config['overlay_intensity'][2],
                log=log
            )
    
        if config['overlay_intensity'][3] > 0:
            out_img[:,:,3],pattern_mask, ssim_value_K = apply_pattern(
                out_img[:,:,3],
                pattern_img,
                m,
                pattern_style['pattern_img_position'],
                config,
                method=row['method'],
                overlay_weight=config['overlay_weight'] * config['overlay_intensity'][3],
                log=log
            )
    
        row['ssim'] = (ssim_value_C + ssim_value_M + ssim_value_Y + ssim_value_K) / 4
        rows_of_image.append(row)
    
        times.append((time(), f'Maske implementiert'))
        log(f'{ times[-1][0] - times[0][0] }: Maske { i } implementiert')

    # bild ablegen
    if len([r for r in rows_of_image if r['ssim'] > config['max_tile_ssim']]) > 0: 
        out_path = f'generic_data/{ get_generic_image_name( page_row, rows_of_image[0]["method"], rows_of_image[0]["idx"] ) }'
        imagefile = BytesIO()
        Image.fromarray(out_img, mode="CMYK").save( imagefile, format='JPEG', dpi=(dotenv['LOFI_DPI'], dotenv['LOFI_DPI']), progressive=True )
    
        # buffer hochladen
        upload_buffer(
            imagefile.getvalue(),
            out_path
        )
    
        # bild in DB ablegen
        save_generic_image(
            rows_of_image
        )

  0%|          | 0/5 [00:00<?, ?it/s]

start
2.2032060623168945: Vorbereitung abgeschlossen
55.68431305885315: Maske 0 geladen
1.4566800594329834: bounding box berechnet
1.45796799659729: tiles berechnet
1.4746208190917969: Anwendung: soft_light abgeschlossen
1.4928300380706787: ssim errechnet: 0.5474598546167395
1.656109094619751: tile eingepast
1.7342867851257324: objektmaske angewendet
57.44359087944031: Maske 0 implementiert
59.43073892593384: Maske 1 geladen
1.4558651447296143: bounding box berechnet
1.4568371772766113: tiles berechnet
1.4582421779632568: Anwendung: soft_light abgeschlossen
1.4613220691680908: ssim errechnet: 0.29985743954306887
1.6113221645355225: tile eingepast
1.691047191619873: objektmaske angewendet
61.138237953186035: Maske 1 implementiert
63.656656980514526: Maske 2 geladen
1.4755208492279053: bounding box berechnet
1.4794957637786865: tiles berechnet
1.8508758544921875: Anwendung: soft_light abgeschlossen
3.125898838043213: ssim errechnet: 0.5791212520641468
3.378919839859009: tile eingepast
3.